In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Raqwan\\Desktop\\ML_DL_Project\\MlOps\\MLFlow_Cancer_Classification\\Coding_Manually\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Raqwan\\Desktop\\ML_DL_Project\\MlOps\\MLFlow_Cancer_Classification\\Coding_Manually'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
# tf.config.run_functions_eagerly(True)
# tf.executing_eagerly()  # Harus return True

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

        # Buat ulang optimizer untuk menghindari error kompatibilitas
        self.optimizer = tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate)  # Sesuaikan learning rate
    
        self.model.compile(
            optimizer=self.optimizer,  # Terapkan optimizer baru
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(rescale=1./255, validation_split=0.20)
        dataflow_kwargs = dict(target_size=self.config.params_image_size[:-1], batch_size=self.config.params_batch_size, interpolation="bilinear")

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data, subset="validation", shuffle=False, **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40, horizontal_flip=True, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data, subset="training", shuffle=True, **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Pastikan eager execution tetap ON (karena tf.data.Dataset butuh eager mode)
        tf.config.run_functions_eagerly(True)  # Pastikan eksekusi tetap dalam Graph Mode

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(path=self.config.trained_model_path, model=self.model)


In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-01-31 14:21:57,502: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-31 14:21:57,505: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-31 14:21:57,508: INFO: common: created directory at: artifacts]
[2025-01-31 14:21:57,509: INFO: common: created directory at: artifacts\training]
[2025-01-31 14:21:57,957: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.


c:\Users\Raqwan\Desktop\ML_DL_Project\MlOps\MLFlow_Cancer_Classification\Coding_Manually\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5


c:\Users\Raqwan\Desktop\ML_DL_Project\MlOps\MLFlow_Cancer_Classification\Coding_Manually\env\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


17/17 ━━━━━━━━━━━━━━━━━━━━ 119s 7s/step - accuracy: 0.5421 - loss: 11.1924 - val_accuracy: 0.8750 - val_loss: 0.1834
Epoch 2/5
 1/17 ━━━━━━━━━━━━━━━━━━━━ 1:47 7s/step - accuracy: 0.7500 - loss: 0.8860

c:\Users\Raqwan\Desktop\ML_DL_Project\MlOps\MLFlow_Cancer_Classification\Coding_Manually\env\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


17/17 ━━━━━━━━━━━━━━━━━━━━ 15s 526ms/step - accuracy: 0.7500 - loss: 0.8860 - val_accuracy: 0.3906 - val_loss: 16.1310
Epoch 3/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 117s 7s/step - accuracy: 0.4951 - loss: 16.0891 - val_accuracy: 0.6094 - val_loss: 8.6107
Epoch 4/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 15s 515ms/step - accuracy: 0.7500 - loss: 8.1347 - val_accuracy: 1.0000 - val_loss: 0.0319
Epoch 5/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 116s 7s/step - accuracy: 0.6033 - loss: 11.4980 - val_accuracy: 0.3906 - val_loss: 31.5532
[2025-01-31 14:28:19,773: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


: 